In [2]:
!pip install open_clip_torch
!pip install easyfsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.1 MB/s eta 0:00:00


In [3]:
import math
import scipy
import timeit
import time
import threading
import csv
import cv2
import joblib
import shutil
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from glob import glob
from tqdm.notebook import tqdm as tqdm
from scipy import stats
from collections import Counter
from sklearn.utils import class_weight

import tensorflow as tf
import tensorflow_addons as tfa
import keras
from tensorflow.keras.utils import load_img, img_to_array
from keras.utils import to_categorical

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Food101, Omniglot
from torchvision.models import resnet18
from PIL import Image
import open_clip

from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average

from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import load_model, Sequential
from keras.applications import InceptionResNetV2
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import LeaveOneOut, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/oxml-carinoma-classification/img_701568.png
/kaggle/input/oxml-carinoma-classification/img_405863.png
/kaggle/input/oxml-carinoma-classification/img_217481.png
/kaggle/input/oxml-carinoma-classification/img_136044.png
/kaggle/input/oxml-carinoma-classification/img_585360.png
/kaggle/input/oxml-carinoma-classification/img_779156.png
/kaggle/input/oxml-carinoma-classification/img_38771.png
/kaggle/input/oxml-carinoma-classification/img_57613.png
/kaggle/input/oxml-carinoma-classification/img_279108.png
/kaggle/input/oxml-carinoma-classification/img_39157.png
/kaggle/input/oxml-carinoma-classification/img_769175.png
/kaggle/input/oxml-carinoma-classification/img_520490.png
/kaggle/input/oxml-carinoma-classification/img_630916.png
/kaggle/input/oxml-carinoma-classification/img_583497.png
/kaggle/input/oxml-carinoma-classification/img_531231.png
/kaggle/input/oxml-carinoma-classification/img_215078.png
/kaggle/input/oxml-carinoma-classification/img_49868.png
/kaggle/input/oxml

In [5]:
pngs_list = glob('/kaggle/input/oxml-carinoma-classification/*png')
df = pd.read_csv('/kaggle/input/oxml-carinoma-classification/labels.csv')
df.head()

,id,malignant
0,441,-1
1,33284,-1
2,38771,1
3,46784,-1
4,57613,-1


In [6]:
source_dir = "/kaggle/input/oxml-carinoma-classification/"

# Destination directories for train and test sets
train_dir = "/kaggle/working/train/"
val_dir = "/kaggle/working/validation/"
test_dir = "/kaggle/working/test/"

In [10]:
# List of IDs for images to be placed in the training set and validation set
train_ids = df['id'].tolist()
val_ids = [441, 918503, 955514, 112905, 38771, 308363]

for i in val_ids:
    if i in train_ids:
        train_ids.remove(i)

# Create the train and test directories if they don't exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Create subdirectories for each class in the train directory
for class_label in [-1, 0, 1]:
    class_subdir = os.path.join(train_dir, str(class_label))
    class_subdir_val = os.path.join(val_dir, str(class_label))
    os.makedirs(class_subdir)
    os.makedirs(class_subdir_val)

# Loop through the PNG photos and move them to the appropriate directory
for filename in tqdm(os.listdir(source_dir)):
    if filename.endswith(".png"):
        # Extract the image ID and label
        image_id = int(filename.split(".")[0].split("_")[1])

        # Check if the image ID is in the train set
        if image_id in train_ids:
            label = df.loc[df['id'] == image_id, 'malignant'].item()
            class_subdir = os.path.join(train_dir, str(label))
            shutil.copy(os.path.join(source_dir, filename), class_subdir)
        elif image_id in val_ids:
            label = df.loc[df['id'] == image_id, 'malignant'].item()
            class_subdir_val = os.path.join(val_dir, str(label))
            shutil.copy(os.path.join(source_dir, filename), class_subdir_val)
        else:
            shutil.copy(os.path.join(source_dir, filename), test_dir)


  0%|          | 0/187 [00:00<?, ?it/s]

In [8]:
import torch
from PIL import Image
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preprocess(Image.open("/kaggle/input/oxml-carinoma-classification/img_38771.png")).unsqueeze(0)
text = tokenizer(['HES stained histopathological slice carcinoma negative',
                  'HES stained histopathological slice carcinoma benign',
                  'HES stained histopathological slice carcinoma malignant'])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", np.argmax(text_probs).item())  # prints: [[1., 0., 0.]]

Label probs: 0


In [11]:
image_ids = []
predicted_labels = []

# Get the list of PNG image file names in the test directory
test_files = glob('/kaggle/working/test/*')
for filename in test_files:
    image_id = int(filename.split(".")[0].split("_")[1])
    image_ids.append(image_id)
    
text = tokenizer(['carcinoma negative','carcinoma benign','carcinoma malignant'])
#text = tokenizer(['negative','benign','malignant '])

for file in tqdm(test_files):
    # Predict the label
    image = preprocess(Image.open(file)).unsqueeze(0)

    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    
    predicted_class = torch.argmax(text_probs).item()
    predicted_labels.append(predicted_class - 1)

# Create a DataFrame with the image IDs and predicted labels
results_df = pd.DataFrame({'id': image_ids, 'malignant': predicted_labels}) 

  0%|          | 0/124 [00:00<?, ?it/s]

In [12]:
print(results_df['malignant'].value_counts())
results_df

-1    124
Name: malignant, dtype: int64


,id,malignant
0,697673,-1
1,346084,-1
2,175012,-1
3,35946,-1
4,666409,-1
...,...,...
119,37944,-1
120,150775,-1
121,439840,-1
122,245987,-1


In [13]:
results_df.to_csv("/kaggle/working/submission.csv", index=False) 